<a href="https://colab.research.google.com/github/ZijinDai/SOSC314-PROJECT/blob/main/CNN_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 49.7 MB/s eta 0:00:00


In [ ]:
# 抓取 CNN 频道中标题含 ICE 的视频（忽略大小写）
# 注意：抓取大型频道建议增加 --max-downloads 限制测试数量，例如 --max-downloads 10
!yt-dlp "https://www.youtube.com/@CNN/videos" \
    --match-filter "title ~= '(?i)ICE'" \
    --skip-download \
    --write-info-json \
    --write-comments \
    --output "/content/cnn_ice_data/%(upload_date)s_%(title)s.%(ext)s"

In [ ]:
!yt-dlp "https://www.youtube.com/@CNN/videos" \
    --dateafter 20250101 \
    --playlist-end 100 \
    --match-filter "title ~= '(?i)ICE'" \
    --skip-download \
    --write-info-json \
    --write-comments \
    --write-auto-subs \
    --sub-lang "en.*" \
    --output "/content/cnn_data/%(title)s.%(ext)s"

[youtube:tab] Extracting URL: https://www.youtube.com/@CNN/videos
[youtube:tab] @CNN/videos: Downloading webpage
[download] Downloading playlist: CNN - Videos
[youtube:tab] UCupvZG-5ko_eiXAupbDfxWw page 1: Downloading API JSON
[youtube:tab] UCupvZG-5ko_eiXAupbDfxWw page 2: Downloading API JSON
[youtube:tab] UCupvZG-5ko_eiXAupbDfxWw page 3: Downloading API JSON
[info] Writing playlist metadata as JSON to: /content/cnn_data/CNN - Videos.info.json
[youtube:tab] Playlist CNN - Videos: Downloading 100 items
[download] Federal prosecutors subpoena five Minnesota officials as DOJ defends enforcement operation does not pass filter (title ~= '(?i)ICE'), skipping ..
[download] Air Force One turns back minutes after leaving for Switzerland does not pass filter (title ~= '(?i)ICE'), skipping ..
[download] We picked 23 Innovation Award winners — here are 3 does not pass filter (title ~= '(?i)ICE'), skipping ..
[download] ‘This is crazy’: Macron rips Trump’s tariff threat in Davos does not pass filt

In [ ]:
!yt-dlp "https://www.youtube.com/@CNN/videos" \
    --lazy-playlist \
    --dateafter 20250101 \
    --match-filter "title ~= '(?i)\bICE\b'" \
    --skip-download \
    --write-info-json \
    --write-comments \
    --write-auto-subs \
    --sub-lang "en.*" \
    --output "/content/cnn_data/%(title)s.%(ext)s"

流式输出内容被截断，只能显示最后 5000 行内容。
[download] IVORY COAST ELECTION 5 does not pass filter (title ~= '(?i)\bICE\b'), skipping ..
[download] TX:GOV PERRY PRSR - MAYOR ON THE PLANT OWNER does not pass filter (title ~= '(?i)\bICE\b'), skipping ..
[download] FILE:RASCAL FLATTS TO JOIN TORNADO BENEFIT CONCERT does not pass filter (title ~= '(?i)\bICE\b'), skipping ..
[download] TX:FERTILIZER BLAST MEMORIAL-OBAMA-BOSTON PRAYERS does not pass filter (title ~= '(?i)\bICE\b'), skipping ..
[download] OBAMA:FERGUSON SHOOTING-REPORTERS ARRESTED does not pass filter (title ~= '(?i)\bICE\b'), skipping ..
[download] SCOTUS PROP 8- ANTI-GAY PROTEST SIGN READS 'LOVE' does not pass filter (title ~= '(?i)\bICE\b'), skipping ..
[download] PAUL AUTHORITY does not pass filter (title ~= '(?i)\bICE\b'), skipping ..
[youtube:tab] UCupvZG-5ko_eiXAupbDfxWw page 1727: Downloading API JSON
[download] MIDEAST: GAZA HOMES DESTROYED IN ATTACK does not pass filter (title ~= '(?i)\bICE\b'), skipping ..
[download] TX:DA MURDERED

In [ ]:
!pip install webvtt-py

In [ ]:
import json
import glob
import os
import pandas as pd
import webvtt

def clean_vtt(vtt_path):
    if not os.path.exists(vtt_path): return ""
    try:
        vtt = webvtt.read(vtt_path)
        lines = []
        for line in vtt: lines.extend(line.text.strip().splitlines())
        res, prev = [], None
        for l in lines:
            if l != prev: res.append(l); prev = l
        return "\n".join(res)
    except: return ""

video_list = []
comment_list = []

data_folder = "/content/cnn_data"
json_files = glob.glob(os.path.join(data_folder, "*.info.json"))

for json_path in json_files:
    with open(json_path, 'r', encoding='utf-8') as f:
        meta = json.load(f)

    v_id = meta.get('id') # Unique video ID

    # 1. Video metadata and subtitles
    vtt_files = glob.glob(json_path.replace(".info.json", "*.vtt"))
    transcript = clean_vtt(vtt_files[0]) if vtt_files else ""

    video_list.append({
        "video_id": v_id,
        "title": meta.get('title'),
        "upload_date": meta.get('upload_date'),
        "channel": meta.get('channel'),
        "view_count": meta.get('view_count'),
        "description": meta.get('description'),
        "transcript": transcript
    })

    # 2. Comments
    comments = meta.get('comments', [])
    for c in comments:
        # 1. Extract the original timestamp
        raw_ts = c.get('timestamp')

        # 2. Convert timestamp to a numeric format, and then to a date format
        try:
            if raw_ts:
                readable_ts = pd.to_datetime(pd.to_numeric(raw_ts), unit='s')
            else:
                readable_ts = None
        except Exception:
            readable_ts = None

        comment_list.append({
            "video_id": v_id,
            "author": c.get('author'),
            "comment_text": c.get('text'),
            "timestamp": readable_ts,
            "like_count": c.get('like_count')
        })

# --- Save as double sheets using Pandas' ExcelWriter ---
output_file = "CNN_Data.xlsx"
with pd.ExcelWriter(output_file) as writer:
    pd.DataFrame(video_list).to_excel(writer, sheet_name='Video_Summary', index=False)
    pd.DataFrame(comment_list).to_excel(writer, sheet_name='Comment_Details', index=False)

print(f"Double sheets completed: {output_file}")

Double sheets completed: CNN_Data.xlsx
